In [ ]:
!pip install -U bitsandbytes accelerate


In [ ]:
!pip install -q peft transformers accelerate bitsandbytes


In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes sentencepiece huggingface_hub


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import files
uploaded = files.upload()

import json
from datasets import Dataset

data = [json.loads(line) for line in open("mcq_generator_dataset.jsonl")][:500]
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    prompt = f"""### Instruction:
Generate a multiple choice question with 4 options and one answer based on the paragraph below.

Paragraph: {example['instruction']}

### Response:
{example['response']}"""
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    prompt = f"""### Instruction:
Generate a multiple choice question with 4 options and one answer based on the paragraph below.

Paragraph: {example['instruction']}

### Response:
{example['response']}"""
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


In [ ]:
model.push_to_hub("Lingesh-S/mcq-mistral-lora")
tokenizer.push_to_hub("Lingesh-S/mcq-mistral-lora")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

base_model = "mistralai/Mistral-7B-Instruct-v0.1"
adapter_repo = "Lingesh-S/mcq-mistral-lora"
offload_folder = "./offload"

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder=offload_folder
)

model = PeftModel.from_pretrained(model, adapter_repo)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = """
# Instruction:
Generate a multiple choice question with 4 options and one correct answer based on the paragraph below.

Paragraph: The Lok Sabha is the House of the People in India. It is one of the two houses of Parliament.

# Response:"""

output = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.5)
print(output[0]["generated_text"])
